In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision 
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm


In [2]:
def conv(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 3, stride = 1, padding = 'same', bias = False)

def pool(): 
    return nn.AvgPool2d(kernel_size = 2, stride = 2)

def conv1x1(channels_in, channels_out): 
    return nn.Conv2d(channels_in, channels_out, kernel_size = 1, stride = 1, padding = 'same')

def bn(channels_in): 
    return nn.BatchNorm2d(channels_in)

def relu():
    return nn.ReLU()

def dropout(rate):
    return nn.Dropout(rate)

In [3]:
class Layer(nn.Sequential):
    def __init__(self, channels):
        super(Layer, self).__init__()
        self.add_module('norm', bn(channels))
        self.add_module('relu', relu())
        self.add_module('conv', conv(channels, channels))
    

In [4]:
class ResBlock(nn.Module):
    def __init__(self, channels, number_of_layers):
        super(ResBlock, self).__init__()
        for i in range(number_of_layers):
            layer = Layer(channels = channels)
            self.add_module('layer%d' % (i + 1), layer)
    
    def forward(self, x):
        for name, layer in self.named_children():
            if name == 'layer1':
                features = layer(x)
            else: 
                features = layer(x + old_features) # += operator does not work, Bug source.
            
            old_features = features 
        return features

In [5]:
class Transition_Layer(nn.Sequential):
    def __init__(self, channels_in, channels_out):
        super(Transition_Layer, self).__init__()
        self.add_module('norm', bn(channels_in)),
        self.add_module('relu', nn.ReLU()),
        self.add_module('conv1x1', conv1x1(channels_in, channels_out)),
        self.add_module('pool', pool())
        self.add_module('dropout', dropout(0.1))

In [6]:
class stage(nn.Sequential):
    def __init__(self, channels_in, channels_out, number_of_layers):
        super(stage, self).__init__()
        self.add_module('res_block', ResBlock(channels_in, number_of_layers))
        self.add_module('transition_layer', Transition_Layer(channels_in, channels_out))

In [7]:
class ResNet(nn.Sequential):
    def __init__(self):
        super(ResNet, self).__init__()
        self.add_module('stage1', stage(3, 8, 3)) # 32
        self.add_module('stage2', stage(8, 16, 3)) # 16
        self.add_module('stage3', stage(16, 32, 3)) # 8
        self.add_module('stage4', stage(32, 32, 3)) # 4
        self.add_module('stage5', stage(32, 16, 3)) # 2
        self.add_module('res_block_final', ResBlock(16, 3)) # 1
        self.add_module('conv1x1', conv1x1(16, 10)) 
        self.add_module('flatten', nn.Flatten())

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
model = ResNet()

In [10]:
model = model.to(device)

In [11]:
# Hyperparameters
learning_rate = 3e-4
batch_size = 128
num_epochs = 30

In [12]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(
         (0.5, 0.5, 0.5),
         (0.5, 0.5, 0.5)
     )])

trainset = torchvision.datasets.CIFAR10(
                root = './data',
                train = True,
                download = True,
                transform = transform
            )
train_loader = torch.utils.data.DataLoader(
                trainset,
                batch_size = batch_size,
                shuffle = True,
                num_workers = 2
            )

testset = torchvision.datasets.CIFAR10(
                root = './data',
                train = False,
                download = True,
                transform = transform
            )
test_loader = torch.utils.data.DataLoader(
                testset,
                batch_size = batch_size,
                shuffle = False,
                num_workers = 2
            )

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 1e-5)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr = 0.1,
    steps_per_epoch = len(train_loader),
    epochs = num_epochs
)

In [14]:
from torchinfo import summary

model = model
summary(model, input_size=(batch_size, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [128, 10]                 --
├─stage: 1-1                             [128, 8, 16, 16]          --
│    └─ResBlock: 2-1                     [128, 3, 32, 32]          --
│    │    └─Layer: 3-1                   [128, 3, 32, 32]          87
│    │    └─Layer: 3-2                   [128, 3, 32, 32]          87
│    │    └─Layer: 3-3                   [128, 3, 32, 32]          87
│    └─Transition_Layer: 2-2             [128, 8, 16, 16]          --
│    │    └─BatchNorm2d: 3-4             [128, 3, 32, 32]          6
│    │    └─ReLU: 3-5                    [128, 3, 32, 32]          --
│    │    └─Conv2d: 3-6                  [128, 8, 32, 32]          32
│    │    └─AvgPool2d: 3-7               [128, 8, 16, 16]          --
│    │    └─Dropout: 3-8                 [128, 8, 16, 16]          --
├─stage: 1-2                             [128, 16, 8, 8]           --
│    └─ResBlock:

In [15]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)
        
        
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    print(f'Epoch:{epoch+1}, Loss:{loss.item():f}')
    scheduler.step()


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:44<00:00,  8.81it/s]


Epoch:1, Loss:1.598669


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:48<00:00,  7.98it/s]


Epoch:2, Loss:1.574424


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:47<00:00,  8.27it/s]


...

Epoch:28, Loss:1.000506


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:35<00:00, 10.90it/s]


Epoch:29, Loss:0.761227


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:35<00:00, 11.07it/s]

Epoch:30, Loss:0.953767


In [16]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 74.12
Accuracy on test set: 67.76
